In [1]:
import sys
sys.path.append("..")

import json
import numpy as np

In [2]:
def get_results(filename):
    with open(filename, "r") as f:
        data = json.load(f)
    #print("Number of data points: ", len(data))
    return data

In [3]:
def get_test_data_annotations():
    visual_news_data = json.load(open("../../datasets/visualnews/origin/data.json"))
    visual_news_data_mapping = {ann["id"]: ann for ann in visual_news_data}

    test_data = json.load(open("../../news_clippings/news_clippings/data/merged_balanced/test.json"))
    annotations = test_data["annotations"]
    return annotations
test_data_annotations = get_test_data_annotations()
test_data_annotations = test_data_annotations[:1000]

In [4]:
def fix_unsures(data):
    """
    In instances where one of the models does not have any output, consider the output of the other model to be the truth
    """
    num_unsures = 0
    for i in range(len(data)):
        if data[i]['falsified'] == "Unsure":
            num_unsures += 1
            if data[i]['output']['model_0'] == "" and data[i]['output']['model_1'] != "":
                if "YES" in data[i]['output']['model_1'] or "Yes" in data[i]['output']['model_1']:
                    data[i]['falsified'] = True
                elif "NO" in data[i]['output']['model_1'] or "No" in data[i]['output']['model_1']:
                    data[i]['falsified'] = False
            elif data[i]['output']['model_0'] != "" and data[i]['output']['model_1'] == "":
                if "YES" in data[i]['output']['model_0'] or "Yes" in data[i]['output']['model_0']:
                    data[i]['falsified'] = True
                elif "NO" in data[i]['output']['model_0'] or "No" in data[i]['output']['model_0']:
                    data[i]['falsified'] = False
    print("Num unsures: ", num_unsures)
    return data

In [5]:
def num_disagreements(data):
    """
    Function to check when models actually disagree and when they are just unsure
    """
    disagreements = 0
    for i in range(len(data)):
        m0 = data[i]['output']['model_0']
        m1 = data[i]['output']['model_1']
        if data[i]['falsified'] == "Unsure":
            #check if models disagree
            if ("YES" in m0 or "Yes" in m0) and ("NO" in m1 or "No" in m1 or "Unsure" in m1):
                disagreements += 1
            elif ("NO" in m0 or "No" in m0) and ("YES" in m1 or "Yes" in m1 or "Unsure" in m1):
                disagreements += 1
            elif "Unsure" in m0 and ("YES" in m1 or "NO" in m1 or "Yes" in m1 or "No" in m1):
                disagreements += 1
    print("Num disagreements: ", disagreements)

In [6]:
def get_accuracy(data, annotations):
    num_correct = 0
    data = fix_unsures(data)
    incorrect_idx = []
    for i in range(len(data)):
        if bool(data[i]['falsified']) == annotations[i]['falsified']:
            num_correct += 1
        else:
            incorrect_idx.append(i)
    print("Num incorrects: ", len(incorrect_idx))
    return num_correct/len(data)

In [7]:
def get_incorrect_idx(data, annotations):
    num_correct = 0
    data = fix_unsures(data)
    incorrect_idx = []
    for i in range(len(data)):
        if bool(data[i]['falsified']) != annotations[i]['falsified']:
            incorrect_idx.append(i)
    return incorrect_idx

In [8]:
def get_acc_without_unsures(data, annotations):
    num_correct, num_unsures = 0,0
    for i in range(len(data)):
        if data[i]['falsified'] == 'Unsure':
            num_unsures += 1
        elif bool(data[i]['falsified']) == annotations[i]['falsified']:
            num_correct += 1
    return num_correct/(len(data) - num_unsures)

In [9]:
def true_positives(data, annotations):
    num_tp = 0
    for i in range(len(data)):
        if annotations[i]['falsified'] == True and bool(data[i]['falsified']) == True:
            num_tp += 1
    return num_tp

In [10]:
def false_positives(data, annotations):
    num_fp = 0
    for i in range(len(data)):
        if annotations[i]['falsified'] == False and bool(data[i]['falsified']) == True:
            num_fp += 1
    return num_fp

In [11]:
def false_negatives(data, annotations):
    num_fn = 0
    for i in range(len(data)):
        if annotations[i]['falsified'] == True and bool(data[i]['falsified']) == False:
            num_fn += 1
    return num_fn

In [12]:
def get_precision_and_recall(data, annotations):
    tp,fp,fn = true_positives(data, annotations), false_positives(data, annotations), false_negatives(data, annotations)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    return precision, recall

### no web access

In [180]:
no_web_file = "../results/results_no_web_access.json"
result_data = get_results(no_web_file)
result_data = result_data[:1000]
num_disagreements(result_data)
precision, recall = get_precision_and_recall(result_data, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data, test_data_annotations)))


Num disagreements:  4
Num unsures:  675
Num incorrects:  428
Accuracy: 0.5720
Precision: 0.5448
Recall: 0.8640
Accuracy without unsures: 0.7515


### with web access (only when model unsure)

In [38]:
web_access_file = "../results/results_web_access_no_initial_context.json"
result_data2 = get_results(web_access_file)
num_disagreements(result_data2)
precision, recall = get_precision_and_recall(result_data2, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data2, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data2, test_data_annotations)))

Num disagreements:  16
Num unsures:  329
Accuracy: 0.5960
Precision: 0.5804
Recall: 0.6860
Accuracy without unsures: 0.6433


In [41]:
web_access_file = "../results/results_initial_context.json"
result_data3 = get_results(web_access_file)
num_disagreements(result_data3)
precision, recall = get_precision_and_recall(result_data3, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data3, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data3, test_data_annotations)))

Num disagreements:  7
Num unsures:  9
Accuracy: 0.8580
Precision: 0.8266
Recall: 0.9060
Accuracy without unsures: 0.8587


### LLaVA not human baseline

In [14]:
web_access_file = "../../datasets/results/llava_not_human.json"
result_data3 = get_results(web_access_file)
num_disagreements(result_data3)
precision, recall = get_precision_and_recall(result_data3, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data3, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data3, test_data_annotations)))

Num disagreements:  39
Num unsures:  48
Num incorrects:  247
Accuracy: 0.7530
Precision: 0.6841
Recall: 0.9400
Accuracy without unsures: 0.7712


### with disambiguation queries

In [16]:
web_access_file = "../results/results_with_disambiguation.json"
result_data4 = get_results(web_access_file)
num_disagreements(result_data4)
precision, recall = get_precision_and_recall(result_data4, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data4, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data4, test_data_annotations)))

Num disagreements:  5
Num unsures:  12
Accuracy: 0.7730
Precision: 0.7468
Recall: 0.8260
Accuracy without unsures: 0.7786


### with opposite stances and disambiguation queries

In [ ]:
web_access_file = "../scripts/temp_final_disamb.json"
result_data5 = get_results(web_access_file)
num_disagreements(result_data5)
precision, recall = get_precision_and_recall(result_data5, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data5, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data5, test_data_annotations)))

In [24]:
web_access_file = "../scripts/disamb_res_updated_1.json"
result_data5 = get_results(web_access_file)
print(len(result_data5))
num_disagreements(result_data5)
precision, recall = get_precision_and_recall(result_data5, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data5, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data5, test_data_annotations)))

253
Num disagreements:  1
Num unsures:  6
Accuracy: 0.7352
Precision: 0.7153
Recall: 0.7778
Accuracy without unsures: 0.7368


### actor-skeptic setup

In [15]:
web_access_file = "../results/results_actor_skeptic.json"
result_data6 = get_results(web_access_file)
print(len(result_data6))
#num_disagreements(result_data6)
precision, recall = get_precision_and_recall(result_data6, test_data_annotations)
#print("Accuracy: {:.4f}".format(get_accuracy(result_data6, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data6, test_data_annotations)))

1000
Precision: 0.6610
Recall: 0.6940
Accuracy without unsures: 0.6945


### Finetuned model results

In [18]:
web_access_file = "../results/results_finetuned.json"
result_data7 = get_results(web_access_file)
print(len(result_data7))
num_disagreements(result_data7)
precision, recall = get_precision_and_recall(result_data7, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data7, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data7, test_data_annotations)))

1000
Num disagreements:  8
Num unsures:  25
Num incorrects:  182
Accuracy: 0.8180
Precision: 0.7449
Recall: 0.9580
Accuracy without unsures: 0.8192


In [19]:
web_access_file = "../results/results_finetuned_debate_refined.json"
result_data7 = get_results(web_access_file)
print(len(result_data7))
num_disagreements(result_data7)
precision, recall = get_precision_and_recall(result_data7, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data7, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data7, test_data_annotations)))

1000
Num disagreements:  40
Num unsures:  234
Num incorrects:  451
Accuracy: 0.5490
Precision: 0.5180
Recall: 0.9500
Accuracy without unsures: 0.5912


### Debating with judge

In [13]:
def get_accuracy(data, annotations):
    num_correct = 0
    incorrect_idx = []
    for i in range(len(data)):
        if bool(data[i]['falsified']) == annotations[i]['falsified']:
            num_correct += 1
        else:
            incorrect_idx.append(i)
    print("Num incorrects: ", len(incorrect_idx))
    return num_correct/len(data)

In [19]:
web_access_file = "../scripts/dj_temp.json"
result_data7 = get_results(web_access_file)
print(len(result_data7))
#num_disagreements(result_data7)
precision, recall = get_precision_and_recall(result_data7, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data7, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data7, test_data_annotations)))

27
Num unsures:  0
Num incorrects:  9
Accuracy: 0.6667
Precision: 0.6667
Recall: 0.6154
Accuracy without unsures: 0.6667


In [18]:
web_access_file = "../../datasets/results/gpt4o_res_temp_final.json"
result_data7 = get_results(web_access_file)
print(len(result_data7))
#num_disagreements(result_data7)
precision, recall = get_precision_and_recall(result_data7, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data7, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data7, test_data_annotations)))

216
Num unsures:  2
Num incorrects:  20
Accuracy: 0.9074
Precision: 0.8729
Recall: 0.9537
Accuracy without unsures: 0.9159


In [28]:
web_access_file = "../../datasets/results/gpt4o_final/final_compiled.json"
result_data7 = get_results(web_access_file)
print(len(result_data7))
#num_disagreements(result_data7)
precision, recall = get_precision_and_recall(result_data7, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data7, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data7, test_data_annotations)))

1000
Num unsures:  0
Num incorrects:  141
Accuracy: 0.8590
Precision: 0.7928
Recall: 0.9720
Accuracy without unsures: 0.8590


In [32]:
web_access_file = "../../datasets/results/gpt4o_final/corrected_results.json"
result_data7 = get_results(web_access_file)
print(len(result_data7))
#num_disagreements(result_data7)
precision, recall = get_precision_and_recall(result_data7, test_data_annotations)
print("Accuracy: {:.4f}".format(get_accuracy(result_data7, test_data_annotations)))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy without unsures: {:.4f}".format(get_acc_without_unsures(result_data7, test_data_annotations)))

1000
Num unsures:  17
Num incorrects:  92
Accuracy: 0.9080
Precision: 0.8505
Recall: 0.9900
Accuracy without unsures: 0.9227


### Analysing where model fails (what kinds of examples the model fails on)

In [22]:
from PIL import Image
from utils.data import get_data, show_data

In [23]:
def retrieve_summary(key):
    with open("../utils/summaries.json", "r") as f:
        data = json.load(f)
    return data[key]

In [24]:
def failed_samples(incorrect_idx, res_data):
    for i in incorrect_idx:
        img, caption, _, annotation = get_data(i)
        display(img)
        show_data(i)
        key = str(annotation['id'])+"_"+str(annotation['image_id'])
        print("Associated summary: ", retrieve_summary(key))
        print("Model_prediction: ", res_data[i]['falsified'])
        print("Model arguments: ", res_data[i]['output'])
        cont = input()
        if "exit" == cont:
            break

In [30]:
incorrect_idx = get_incorrect_idx(result_data7, test_data_annotations)
print(incorrect_idx)
#failed_samples(incorrect_idx, result_data7)

Num unsures:  0
[10, 14, 31, 34, 38, 62, 74, 80, 110, 120, 126, 151, 152, 164, 175, 199, 205, 216, 218, 224, 226, 234, 241, 242, 246, 250, 256, 260, 272, 284, 286, 300, 304, 320, 334, 336, 338, 348, 354, 360, 362, 372, 384, 386, 389, 398, 400, 404, 406, 410, 412, 416, 428, 436, 438, 446, 460, 464, 466, 470, 473, 480, 482, 496, 498, 502, 510, 516, 518, 520, 528, 530, 535, 538, 554, 562, 566, 580, 584, 588, 598, 600, 602, 608, 614, 623, 626, 651, 660, 666, 668, 672, 676, 678, 684, 686, 700, 708, 719, 720, 722, 726, 728, 752, 760, 762, 770, 772, 774, 778, 780, 786, 790, 794, 806, 814, 823, 826, 832, 836, 838, 842, 854, 856, 860, 864, 872, 874, 882, 888, 890, 900, 932, 940, 952, 962, 965, 972, 974, 978, 990]


In [31]:
len(incorrect_idx)

141